<a href="https://colab.research.google.com/github/Sreeya22/Enhancing-Crop-Health-Image-Classification-of-Plant-Diseases/blob/main/leaf_detect_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.models import Sequential
#initialize nn

from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
#convert pooling features space to large feature vector for fully
#connected layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense

from keras.layers import BatchNormalization
from keras.layers import Dropout

#basic cnn
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(128,128, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(25, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])



train_datagen = ImageDataGenerator(rescale = None,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/AEC dataset/dataset/train',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
#print(test_datagen);
labels = (training_set.class_indices)
print(labels)

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/AEC dataset/dataset/test',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

labels2 = (test_set.class_indices)
print(labels2)

model.fit(training_set,
                         steps_per_epoch = 375,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 125)


# Part 3 - Making new predictions

model_json=model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
    model.save_weights("model1.weights.h5")
    print("Saved model to disk")

Found 375 images belonging to 25 classes.
{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___Healthy': 3, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 4, 'Corn_(maize)___Common_rust': 5, 'Corn_(maize)___Healthy': 6, 'Corn_(maize)___Northern_Leaf_Blight': 7, 'Grape___Black_rot': 8, 'Grape___Esca_(Black_Measles)': 9, 'Grape___Healthy': 10, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 11, 'Potato___Early_blight': 12, 'Potato___Healthy': 13, 'Potato___Late_blight': 14, 'Tomato___Bacterial_spot': 15, 'Tomato___Early_blight': 16, 'Tomato___Healthy': 17, 'Tomato___Late_blight': 18, 'Tomato___Leaf_Mold': 19, 'Tomato___Septoria_leaf_spot': 20, 'Tomato___Spider_mites Two-spotted_spider_mite': 21, 'Tomato___Target_Spot': 22, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 23, 'Tomato___Tomato_mosaic_virus': 24}
Found 127 images belonging to 25 classes.
{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___Healthy': 3, 

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
import cv2
import os

# Load the model from JSON and weights
json_file_path = 'model1.json'
weights_file_path = 'model1.weights.h5'

# Check if the model files exist
if not os.path.isfile(json_file_path) or not os.path.isfile(weights_file_path):
    raise FileNotFoundError("Model files not found. Check the file paths.")

# Load model architecture and weights
with open(json_file_path, 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(weights_file_path)
print("Loaded model from disk")

# Labels for the model
label = ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust", "Apple___Healthy",
         "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot", "Corn_(maize)___Common_rust_",
         "Corn_(maize)___Healthy", "Corn_(maize)___Northern_Leaf_Blight", "Grape___Black_rot",
         "Grape___Esca_(Black_Measles)", "Grape___Healthy", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
         "Potato___Early_blight", "Potato___Healthy", "Potato___Late_blight", "Tomato___Bacterial_spot",
         "Tomato___Early_blight", "Tomato___Healthy", "Tomato___Late_blight", "Tomato___Leaf_Mold",
         "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite", "Tomato___Target_Spot",
         "Tomato___Tomato_Yellow_Leaf_Curl_Virus", "Tomato___Tomato_mosaic_virus"]

# Function to preprocess and predict an image
def preprocess_and_predict(image_path, model, label_list):
    # Load and preprocess image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Predict class
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)

    # Ensure the predicted class index is valid
    if predicted_class >= len(label_list):
        raise IndexError("Predicted class index is out of range for the label list.")

    return label_list[predicted_class]

# Test image 1
test_image_path1 = '/content/drive/MyDrive/AEC dataset/im_for_testing_purpose/t.lateblight.JPG'
predicted_label1 = preprocess_and_predict(test_image_path1, loaded_model, label)
print(f"Predicted label for image 1: {predicted_label1}")

# Test image 2
test_image_path2 = '/content/drive/MyDrive/AEC dataset/im_for_testing_purpose/t.yelloleafcurls.JPG'

# Check if the second image file exists
if not os.path.isfile(test_image_path2):
    raise FileNotFoundError(f"The file {test_image_path2} does not exist.")

# Load and preprocess image using OpenCV
test_image = cv2.imread(test_image_path2)
if test_image is None:
    raise ValueError("Image file could not be loaded. Check the file path and format.")

test_image = cv2.resize(test_image, (128, 128))  # Resize to match the model input size
test_image = test_image / 255.0  # Normalize pixel values to [0, 1]
test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

# Predict class
prediction = loaded_model.predict(test_image)
predicted_class = np.argmax(prediction)

# Define labels for this classification
label2 = ["cats", "dogs", "horse", "rose"]

# Print predicted class index for debugging
print(f"Predicted class index for image 2: {predicted_class}")

Loaded model from disk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Predicted label for image 1: Tomato___Spider_mites Two-spotted_spider_mite
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class index for image 2: 19


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

# Define the enhanced CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(25, activation='softmax'))

# Compile the model with an adjusted learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation with rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/AEC dataset/dataset/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/AEC dataset/dataset/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
model.fit(
    training_set,
    steps_per_epoch=2,
    epochs=2,
    validation_data=test_set,
    validation_steps=50,
    verbose=2  # Change verbosity to get more detailed output
)

# Save the model
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model1.weights.h5")
print("Saved model to disk")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 375 images belonging to 25 classes.
Found 127 images belonging to 25 classes.
Epoch 1/2
2/2 - 12s - 6s/step - accuracy: 0.0469 - loss: 9.0332 - val_accuracy: 0.0472 - val_loss: 8.8082
Epoch 2/2
2/2 - 4s - 2s/step - accuracy: 0.0000e+00 - loss: 9.3554 - val_accuracy: 0.0394 - val_loss: 8.8037
Saved model to disk
